In [ ]:
system_prompt = 'You are a programmer,you use python,sympy to solve problems,set variables,exprestions,set Interval to give range of variable,and use solve() function to calculation.only give a python code'

### load data

In [3]:
import json
from datasets import load_dataset, Dataset
with open('conic10k/test.json', 'r', encoding='utf-8') as file:
    raw_conic_tr = json.load(file)
datasett = load_dataset('json', data_files='conic10k/test.json', split=None)
# dataset1['train']['text']
# len(dataset1)
datasett['train'][0]['text']

'如果双曲线的两个焦点分别为$F_{1}(-3,0)$, $F_{2}(3,0)$ ,一条渐近线方程为$y=\\sqrt{2} x$，则该双曲线的方程为?'

### prompt to have answer

In [4]:
#init data

ans = {'Ai_answer':[],'real_answer':[],'primary_output':[]}
flag = 2000
my_dataset = datasett
print(my_dataset)

DatasetDict({
    train: Dataset({
        features: ['query_spans', 'text', 'query_expressions', 'process', 'fact_spans', 'answer_expressions', 'fact_expressions'],
        num_rows: 2069
    })
})


In [5]:
#将大模型输出的结果中的代码提取并运行，将结果返回
import re
import sys
from io import StringIO
def to_py_and_run(text):
    pattern = r'```python\n(.*?)\n```'
    match = re.search(pattern, text, re.DOTALL)
    run_ans = 'None'
    if match:
        code_block = match.group(1).strip()  # 提取代码块并去除首尾空白
        
        # 截取最后一行提取代码结果
        last_line = code_block.rstrip().rsplit('\n', 1)[-1]

        # #最后一行可能是print(answer_expr),也可能是answer_expr，我们归一化处理
        ppattern = r"print\((.*)\)" 
        pmatch = re.search(ppattern, last_line)
        pmatch
        if pmatch:
            no_use=1 # jump
        else:
            code_block = code_block + '\n\nprint('+ last_line+')'
        
        # with open('outputcode.py', 'w') as file:
        #     file.write(code_block)  # 将代码写入文件
        # #为了保存输出，使用重定向


        old_stdout = sys.stdout
        new_stdout = StringIO()
        sys.stdout = new_stdout

        # 运行生成的Python文件 保存输出到返回值，并恢复输出定向
        try:
            exec(code_block,globals())
            output = new_stdout.getvalue()

        except:
            run_ans = 'code_error'
        else:
            # print('printsuccess')
            run_ans = output.strip('\n')
        sys.stdout = old_stdout
            
    return run_ans

In [31]:
ans['primary_output'] = ['empty' for _ in range(73)]
# ans['primary_output']

In [ ]:
# 第73个数据开始，保存了primaryoutput

In [8]:
from gradio_client import Client
client = Client("Qwen/Qwen2-72B-Instruct")
i=flag
while i < len(my_dataset['train']['text']):
  mathproblem = my_dataset['train'][i]['text']
  problemans = my_dataset['train'][i]['answer_expressions']
  result = client.predict(
  query=mathproblem,
  history=[],
  system="You are a programmer,you use python,sympy to solve problems,set variables,set Interval to calculate the range of question,exprestions,and use solve() function to calculation.only give a python code.(code cannot have external inputs and graphical outputs)",
  api_name="/model_chat"
  )
  print('finish predict')
  ans['primary_output'].append(result[1][0][1])
  code_result = to_py_and_run(result[1][0][1])
  if code_result =='code_error':
    stop_point = 0
    stop_limit = 1
    while code_result =='code_error' and stop_point < stop_limit:
      stop_point = stop_point+1
      print('error time:',stop_point)
      new_result = client.predict(
        query='you gave me a code to solve a problem,code:'+code_result+'problem:'+mathproblem+'pleas debug the code,then only give me a corrected python code',
        history=[],
        system="You are a programmer,you use python,sympy to solve problems,set variables,set Interval to calculate the range of question,exprestions,and use solve() function to calculation.only give a python code.(code cannot have external inputs and graphical outputs)",
        api_name="/model_chat"
      )
      code_result = to_py_and_run(new_result[1][0][1])

  ans['Ai_answer'].append(code_result)
  ans['real_answer'].append(problemans)
  print(i)
  print(mathproblem)
  print(code_result)
  print(problemans)
  with open('test_predict_5.json', 'w', encoding='utf-8') as file:
    json.dump(ans, file)
  i = i+1
  flag = i

Loaded as API: https://qwen-qwen2-72b-instruct.hf.space ✔


In [9]:
# query flag
flag

2069

In [10]:
import pandas as pd
# 需要库：openpyxl
# 将字典转换为pandas DataFrame
memory_df = pd.DataFrame(ans)
 
# 将DataFrame写入Excel文件
# !!!!记得修改你的文件名
memory_df.to_excel('qwen2_codeversion_5.xlsx', index=1)